In [ ]:
import requests

resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=XBTUSD')

print(resp.json())

In [7]:
resp.json()["result"]["last"]

1637780400

In [51]:
def get_best_limit_price(pair_name, order_type):
    order_book = get_order_book_response(pair_name)
    if order_type=="buy":
        # {'error': [],
        #'result': {'XXLMZEUR': {'asks': [['0.28805700', '3104.111', 1637781170],
        output = order_book["result"][list(order_book["result"].keys())[0]]["asks"][0][0]
    elif order_type=="sell":
        output = order_book["result"][list(order_book["result"].keys())[0]]["bids"][0][0]
    return float(output)
def get_order_book_response(pair_name, since=None, count=10):
    resp = requests.get(f"https://api.kraken.com/0/public/Depth?pair={pair_name}&since={since}&count={count}")
    return resp.json()

In [54]:
buy_price=get_best_limit_price("xlmeur", "buy")
sell_price=get_best_limit_price("xlmeur", "sell")
print(buy_price)
print(sell_price)
get_order_book_response("xlmeur")

0.288665
0.288509


{'error': [],
 'result': {'XXLMZEUR': {'asks': [['0.28866500', '3675.000', 1637781715],
    ['0.28867700', '1901.829', 1637781715],
    ['0.28867800', '4903.003', 1637781715],
    ['0.28867900', '4823.994', 1637781715],
    ['0.28868000', '3675.000', 1637781714],
    ['0.28869300', '1731.216', 1637781715],
    ['0.28869400', '2455.359', 1637781714],
    ['0.28874100', '1428.373', 1637781714],
    ['0.28876500', '2589.573', 1637781714],
    ['0.28876600', '1720.481', 1637781714]],
   'bids': [['0.28850900', '1732.759', 1637781715],
    ['0.28837900', '1426.685', 1637781715],
    ['0.28837300', '1737.180', 1637781715],
    ['0.28837200', '15483.797', 1637781715],
    ['0.28836700', '3000.000', 1637781715],
    ['0.28836600', '270.334', 1637781693],
    ['0.28836200', '453.529', 1637781692],
    ['0.28835800', '1264.713', 1637781714],
    ['0.28835700', '23218.913', 1637781704],
    ['0.28832200', '490.190', 1637781703]]}}}

In [49]:
a

0.28833

In [29]:
 pd.to_datetime(1637781014, unit='s')

Timestamp('2021-11-24 19:10:14')

In [43]:
list(a["result"].keys())[0]

'XXLMZEUR'

In [8]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
class OrderBookRecorder():
    def __init__(self, pair: str):
        self.pair = pair
        self.history_df = {
            "asks": pd.DataFrame(),
            "bids": pd.DataFrame(),
            "trades": pd.DataFrame()
                          }
        self.trade_columns = [
            "price",
            "volume",
            "timestamp",
            "sell_buy",
            "market_limit",
            "misc"
        ]
        self.order_book_columns = [
            "price",
            "volume",
            "timestamp",
        ]
        
    def get_trades_response(self, since=None):
        resp = requests.get(f"https://api.kraken.com/0/public/Trades?pair={self.pair}&since={since}")
        return resp.json()
    
    def get_order_book_response(self, since=None, count=500):
        resp = requests.get(f"https://api.kraken.com/0/public/Depth?pair={self.pair}&since={since}&count={count}")
        return resp.json()
    
    def from_json_to_df(self, resp_json):
        z_pair_name=list(resp_json["result"].keys())[0] #"XXLMZEUR"
        df = pd.DataFrame(data=resp_json["result"][z_pair_name], columns = self.trade_columns)
        return df
    def from_json_to_df_order_book(self, resp_json, ask_or_bid):
        z_pair_name=list(resp_json["result"].keys())[0] #"XXLMZEUR"
        df = pd.DataFrame(data=resp_json["result"][z_pair_name][ask_or_bid], columns = self.order_book_columns)
        return df
        
    
    def update_data(self, current_data: pd.DataFrame, data_type: str):
        self.history_df[data_type] = pd.concat([self.history_df[data_type], current_data]).drop_duplicates().copy()
        #print(f"Data history updated of {data_type}, last trade history with shape:" , self.history_df[data_type].shape[0])
        
    def process_df(self,df: pd.DataFrame):
        """Process dataframe for ask, bids and also trades"""
        # Converting to regular date format
        df["date"] = pd.to_datetime(df['timestamp'], unit='s')
        # Setting date as index
        df = df.set_index(pd.DatetimeIndex(df["date"]))

        # Removing date from columns (already in index)
        df = df.drop("date", axis=1).copy()
        # sort values
        df = df.sort_values(by="date")
        # Right format
        df[["timestamp", "price", "volume"]] = df[["timestamp", "price", "volume"]].astype(float)
        return df
    def record_trades(self,):
        resp_json = self.get_trades_response()
        current_trade_history_df = self.from_json_to_df(resp_json)
        current_trade_history_df = self.process_df(current_trade_history_df)
        self.update_data(current_trade_history_df, "trades")
        
    def record_order_book(self,):
        resp_json = self.get_order_book_response()
        for ask_or_bid in ["asks", "bids"]:
            current_order_book_history_df = self.from_json_to_df_order_book(resp_json, ask_or_bid)
            current_order_book_history_df = self.process_df(current_order_book_history_df)
            self.update_data(current_order_book_history_df, ask_or_bid)
            #self.update_metrics()


In [9]:
o = OrderBookRecorder(pair="xlmeur")

In [10]:
o.record_order_book()

In [12]:
o.history_df["asks"]

,price,volume,timestamp
date,,,
2021-11-11 22:09:27,0.348559,48.585,1.636669e+09
2021-11-11 22:09:42,0.347794,22.868,1.636669e+09
2021-11-11 22:31:04,0.347380,36.325,1.636670e+09
2021-11-12 09:30:42,0.347510,194.000,1.636709e+09
2021-11-12 10:09:10,0.344996,174.428,1.636712e+09
...,...,...,...
2021-11-24 19:04:15,0.290950,4039.150,1.637781e+09
2021-11-24 19:04:15,0.291838,105055.542,1.637781e+09
2021-11-24 19:04:15,0.290052,1245.603,1.637781e+09


In [14]:
o.history_df["bids"]

,price,volume,timestamp
date,,,
2021-07-22 04:30:11,0.191000,145.418,1.626928e+09
2021-07-22 05:02:08,0.192000,157.445,1.626930e+09
2021-07-22 06:04:07,0.193000,145.418,1.626934e+09
2021-07-22 06:04:24,0.193244,200.000,1.626934e+09
2021-07-22 06:12:08,0.194000,157.445,1.626934e+09
...,...,...,...
2021-11-24 19:04:13,0.289072,6437.977,1.637781e+09
2021-11-24 19:04:14,0.289436,5918.155,1.637781e+09
2021-11-24 19:04:15,0.289481,10001.000,1.637781e+09
